In [1]:
from NN_templates import *

In [4]:
import glob
import json

pk_s = sorted(glob.glob('/pscratch/sd/d/dforero/projects/sec-gen-ez/EZmock/emu_train_wn0/*/PK*RSD.dat'))

confs = sorted(glob.glob('/pscratch/sd/d/dforero/projects/sec-gen-ez/EZmock/emu_train_wn0/*/config.json'))


In [5]:
output_s = []
output2_s = []
input_s = []

for i in range(len(pk_s)):
    output_s.append(np.concatenate(np.loadtxt(pk_s[i])[2:21,5:6]))
    output2_s.append(np.concatenate(np.loadtxt(pk_s[i])[2:21,6:]))
    with open(confs[i]) as f:
        conf = json.load(f)
    input_s.append(np.array([conf[0]['redshift'],conf[0]['pdf_base'],conf[0]['dens_scat'],conf[0]['rand_motion'],conf[0]['dens_cut']]))

In [7]:
PS_all = NNParameterModelPCA(input_s,output_s,5,32, pca_components = None,lr=4e-5)
PS_all2 = NNParameterModelPCA(input_s,output2_s,5,16, pca_components = None,lr=4e-5,noise=0.0)

In [ ]:
PS_all.Optimise(training = 290,validation=10,epochs = 400,batch_size = 290)
PS_all2.Optimise(training = 290,validation=10,epochs = 400,batch_size = 290)

In [ ]:
import matplotlib
matplotlib.rcParams['figure.dpi']=200
delt_st = []
for i in range(len(output_s)):
    plt.plot((PS_all.Predict(input_s[i])[0]-output_s[i])/output_s[i],'C0--',alpha=0.3,)
    delt_st.append((PS_all.Predict(input_s[i])[0]-output_s[i])/output_s[i])
    
    
plt.axhline(0.01,color='black',linestyle='--')
plt.axhline(-0.01,color='black',linestyle='--')
plt.ylim(-20,20)

print([np.mean(delt_st),np.std(delt_st)])
print(np.mean(np.abs(delt_st)))